# Naver 채권분석리포트 Crawling

In [ ]:
# PyPDF2나 pdfminer를 찾을 수 없다고 한다면, 아래 주석을 풀고 install을 시도하세요.
# pip install PyPDF2 
# pip install pdfminer.six

# moduel import
import requests
from bs4 import BeautifulSoup
import os
import urllib.request
from PyPDF2 import PdfFileReader
import sys
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams
import io

pdf_list = [] # pdf의 날짜, 증권사, 제목을 담을 빈 리스트 생성
BASE_DIR = os.getcwd() # 현재 작업하고 있는 곳을 BASE_DIR로 정의
REPORTS_DIR = BASE_DIR # REPORTS_DIR = BASE_DIR

# Naver의 채권분석리포트의 마지막 페이지는 116이므로 117까지 for문 돌릴 준비
for i in range(1, 117):
    print(i,'번 페이지에 있는 문서를 가져오는 중입니다.') # 작업상황 확인을 위함 
    url = 'https://finance.naver.com/research/debenture_list.nhn?&page={}'.format(i) # page={}에 i
    response = requests.get(url) # url로 requests를 보냄 
    soup = BeautifulSoup(response.content, 'html.parser') # soup 생성
    reports = soup.find('table', class_='type_1').find_all('tr')[1:] # 찾아야할 content 축소
    
    # for문 준비 
    for td in reports:
        filename = "" # filename 을 만들 빈 문자열 생성, for문이 돌 때마다 초기화 

        # date 코드
        date = td.find('td', class_='date')
        if date is not None: # None 값들이 검사되므로 if문 추가
            date = date.text # text만 추출
            filename += date # filename 에 추가
            filename += '_' # 다음에 들어올 author와의 구분을 위한 _ 추가 

        # author 코드
        author = td.find_all('td') 
        if len(author) == 5: # 길이를 확인해보니, len이 5인 것만 정상적으로 증권사 이름들이 추출됨 
            author = author[1].text
            filename += author
            filename += '_'

        # title 코드 
        title = td.find('td').find('a')
        if title is not None: # None 값들이 검사되므로 if문 추가
            title = title.text
            filename += title # filename 에 추가 
            # computer가 읽을 수 없는 특수문자들을 대체 (후에 정규표현식으로 고칠 것)
            filename = filename.replace('?','').replace('.','').replace('!','').replace('/','')
            filename = filename.replace(':','').replace(';','').replace('=','').replace('+','')
            filename = filename.replace('-','').replace('*','').replace('"','').replace('<','')
            filename = filename.replace('>','').replace('[','').replace(']','')
            filename = filename + '.pdf'
            pdf_list.append(filename) # pdf_list 에 filename을 추가함 

        # pdf 다운받기 코드
        pdf_href = td.find('td', class_='file')
        if pdf_href is not None: # None값이 있기 때문에 if문 추가
            pdf_href = str(pdf_href.find('a')['href'])
            pdf_url = pdf_href
            print(pdf_url) # pdf_url 출력해서 제대로 된 url인지 확인
            
            r = requests.get(pdf_url) # url로 requests 보냄
            naver_reports_path = os.path.join(REPORTS_DIR, filename) # 작업하고 있는 폴더에 파일 추가
            
            with open(naver_reports_path, 'wb') as f: # 파일 쓰기
                f.write(r.content)
                
print(pdf_list)
print("총 가져온 pdf파일 개수: ", len(pdf_list))
                
# pdf -> txt 로 변환하는 함수                
def pdfparser(data):
    ccount = 0
    BASE_DIR = os.getcwd()
    REPORTS_DIR = BASE_DIR
    
    try: # 오류가 발생하는 pdf 들이 존재하기 때문에, try문 사용
        fp = open(data, 'rb')
        rsrcmgr = PDFResourceManager()
        retstr = io.StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        for page in PDFPage.get_pages(fp):
            interpreter.process_page(page)
            data =  retstr.getvalue()
            text_path = os.path.join(REPORTS_DIR, pdf_list[i]+'.txt')
            with open(text_path, 'w', encoding='utf-8') as f:
                f.write(data)

    except AttributeError:
        ccount += 1
        print(pdf_list[i],'는 Error로 인해 pass합니다.')
        print("현재까지의 누적 Errors: ", ccount)
        pass
    
# 이 작업을 통해, pdf_list에서 날짜, 증권사, 제목을 확인할 수 있고,
# pdf 파일을 얻을 수 있다.
# 또한, pdf 파일을 txt로 변환하는 함수까지 정의해 놓은 상태
# 아래에 있는 코드를 실행하면, pdf를 txt로 변환하기 시작합니다.

In [ ]:
# pdf -> txt 로 변환하는 code
for i in range(1, (len(pdf_list)+1)):
    print(i)
    print(pdf_list[i], '를 txt로 변환 중입니다...') 
    pdfparser(pdf_list[i])

## txt 파일들을 하나로 합치고, .csv 파일로 변환

In [34]:
from os import listdir
from os.path import isfile, join

file_list = [f for f in listdir(r'C:\Users\student\Documents\python_basic\paper_implementation\text_data') 
          if isfile(join(r'C:\Users\student\Documents\python_basic\paper_implementation\text_data', f))]
# print(file_list)

In [33]:
pip install mecab_python-0.996_ko_0.9.2_msvc-cp37-cp37m-win_amd64.whl

Note: you may need to restart the kernel to use updated packages.


Requirement 'mecab_python-0.996_ko_0.9.2_msvc-cp37-cp37m-win_amd64.whl' looks like a filename, but the file does not exist


In [35]:
from ekonlpy.sentiment import MPCK
from ekonlpy.tag import Mecab

In [51]:
def text2ngram(text):
    mpck = MPCK()
    bef_tokens = mpck.tokenize(text)
    ngrams = mpck.ngramize(bef_tokens)
    return ngrams

In [63]:
import pandas as pd
import os

BASE_DIR = os.getcwd()
DATA_DIR = 'C:\\Users\\student\\Documents\\python_basic\\paper_implementation\\text_data'
result = pd.DataFrame(columns=['date','firm','content', 'ng'])
count = 0

try:
#     for i in range(5):
    for i in range(1728, len(file_list)):
        print(i,'번째 루프를 돌고있습니다...')
        filename = file_list[i]
        data_path = os.path.join(DATA_DIR, filename)
        split_file = filename.split('_')
        date = '20'+split_file[0]
        split_file[0] = date
        
        df = pd.DataFrame(columns=['date','firm','content', 'ng']) # columns을 가진 빈 데이터 프레임 생성
        raw_df = pd.DataFrame(split_file)

        df['date'] = raw_df.loc[0]
        df['date'] = pd.to_datetime(df['date']) 
        df['firm'] = raw_df.loc[1]

        with open(data_path, 'r', encoding='utf-8') as f:# 파일 쓰기
            read_content = f.read()
            temp_df = pd.DataFrame([read_content])
            n_gram = text2ngram(read_content)
            n_gramdf = pd.DataFrame([n_gram])
            df['content'] = (temp_df.loc[0])
            df['ng'] = (n_gramdf.loc[0])

        result = pd.concat([result, df])
        
except:
    count +=1
    print(count)
    pass

result.to_csv('final.csv', mode='a')
# print(result)

1728 번째 루프를 돌고있습니다...
1729 번째 루프를 돌고있습니다...
1730 번째 루프를 돌고있습니다...
1731 번째 루프를 돌고있습니다...
1732 번째 루프를 돌고있습니다...
1733 번째 루프를 돌고있습니다...
1734 번째 루프를 돌고있습니다...
1735 번째 루프를 돌고있습니다...
1736 번째 루프를 돌고있습니다...
1737 번째 루프를 돌고있습니다...
1738 번째 루프를 돌고있습니다...
1739 번째 루프를 돌고있습니다...
1740 번째 루프를 돌고있습니다...
1741 번째 루프를 돌고있습니다...
1742 번째 루프를 돌고있습니다...
1743 번째 루프를 돌고있습니다...
1744 번째 루프를 돌고있습니다...
1745 번째 루프를 돌고있습니다...
1746 번째 루프를 돌고있습니다...
1747 번째 루프를 돌고있습니다...
1748 번째 루프를 돌고있습니다...
1749 번째 루프를 돌고있습니다...
1750 번째 루프를 돌고있습니다...
1751 번째 루프를 돌고있습니다...
1752 번째 루프를 돌고있습니다...
1753 번째 루프를 돌고있습니다...
1754 번째 루프를 돌고있습니다...
1755 번째 루프를 돌고있습니다...
1756 번째 루프를 돌고있습니다...
1757 번째 루프를 돌고있습니다...
1758 번째 루프를 돌고있습니다...
1759 번째 루프를 돌고있습니다...
1760 번째 루프를 돌고있습니다...
1761 번째 루프를 돌고있습니다...
1762 번째 루프를 돌고있습니다...
1763 번째 루프를 돌고있습니다...
1764 번째 루프를 돌고있습니다...
1765 번째 루프를 돌고있습니다...
1766 번째 루프를 돌고있습니다...
1767 번째 루프를 돌고있습니다...
1768 번째 루프를 돌고있습니다...
1769 번째 루프를 돌고있습니다...
1770 번째 루프를 돌고있습니다...
1771 번째 루프를 돌고있습니다...
1772 번째 루프를 돌고있습니다...
1773 번째 루프

2101 번째 루프를 돌고있습니다...
2102 번째 루프를 돌고있습니다...
2103 번째 루프를 돌고있습니다...
2104 번째 루프를 돌고있습니다...
2105 번째 루프를 돌고있습니다...
2106 번째 루프를 돌고있습니다...
2107 번째 루프를 돌고있습니다...
2108 번째 루프를 돌고있습니다...
2109 번째 루프를 돌고있습니다...
2110 번째 루프를 돌고있습니다...
2111 번째 루프를 돌고있습니다...
2112 번째 루프를 돌고있습니다...
2113 번째 루프를 돌고있습니다...
2114 번째 루프를 돌고있습니다...
2115 번째 루프를 돌고있습니다...
2116 번째 루프를 돌고있습니다...
2117 번째 루프를 돌고있습니다...
2118 번째 루프를 돌고있습니다...
2119 번째 루프를 돌고있습니다...
2120 번째 루프를 돌고있습니다...
2121 번째 루프를 돌고있습니다...
2122 번째 루프를 돌고있습니다...
2123 번째 루프를 돌고있습니다...
2124 번째 루프를 돌고있습니다...
2125 번째 루프를 돌고있습니다...
2126 번째 루프를 돌고있습니다...
2127 번째 루프를 돌고있습니다...
2128 번째 루프를 돌고있습니다...
2129 번째 루프를 돌고있습니다...
2130 번째 루프를 돌고있습니다...
2131 번째 루프를 돌고있습니다...
2132 번째 루프를 돌고있습니다...
2133 번째 루프를 돌고있습니다...
2134 번째 루프를 돌고있습니다...
2135 번째 루프를 돌고있습니다...
2136 번째 루프를 돌고있습니다...
2137 번째 루프를 돌고있습니다...
2138 번째 루프를 돌고있습니다...
2139 번째 루프를 돌고있습니다...
2140 번째 루프를 돌고있습니다...
2141 번째 루프를 돌고있습니다...
2142 번째 루프를 돌고있습니다...
2143 번째 루프를 돌고있습니다...
2144 번째 루프를 돌고있습니다...
2145 번째 루프를 돌고있습니다...
2146 번째 루프

2475 번째 루프를 돌고있습니다...
2476 번째 루프를 돌고있습니다...
2477 번째 루프를 돌고있습니다...
2478 번째 루프를 돌고있습니다...
2479 번째 루프를 돌고있습니다...
2480 번째 루프를 돌고있습니다...
2481 번째 루프를 돌고있습니다...
2482 번째 루프를 돌고있습니다...
2483 번째 루프를 돌고있습니다...
2484 번째 루프를 돌고있습니다...
2485 번째 루프를 돌고있습니다...
2486 번째 루프를 돌고있습니다...
2487 번째 루프를 돌고있습니다...
2488 번째 루프를 돌고있습니다...
2489 번째 루프를 돌고있습니다...
2490 번째 루프를 돌고있습니다...
2491 번째 루프를 돌고있습니다...
2492 번째 루프를 돌고있습니다...
2493 번째 루프를 돌고있습니다...
2494 번째 루프를 돌고있습니다...
2495 번째 루프를 돌고있습니다...
2496 번째 루프를 돌고있습니다...
2497 번째 루프를 돌고있습니다...
2498 번째 루프를 돌고있습니다...
2499 번째 루프를 돌고있습니다...
2500 번째 루프를 돌고있습니다...
2501 번째 루프를 돌고있습니다...
2502 번째 루프를 돌고있습니다...
2503 번째 루프를 돌고있습니다...
2504 번째 루프를 돌고있습니다...
2505 번째 루프를 돌고있습니다...
2506 번째 루프를 돌고있습니다...
2507 번째 루프를 돌고있습니다...
2508 번째 루프를 돌고있습니다...
2509 번째 루프를 돌고있습니다...
2510 번째 루프를 돌고있습니다...
2511 번째 루프를 돌고있습니다...
2512 번째 루프를 돌고있습니다...
2513 번째 루프를 돌고있습니다...
2514 번째 루프를 돌고있습니다...
2515 번째 루프를 돌고있습니다...
2516 번째 루프를 돌고있습니다...
2517 번째 루프를 돌고있습니다...
2518 번째 루프를 돌고있습니다...
2519 번째 루프를 돌고있습니다...
2520 번째 루프

2848 번째 루프를 돌고있습니다...
2849 번째 루프를 돌고있습니다...
2850 번째 루프를 돌고있습니다...
2851 번째 루프를 돌고있습니다...
2852 번째 루프를 돌고있습니다...
2853 번째 루프를 돌고있습니다...
2854 번째 루프를 돌고있습니다...
2855 번째 루프를 돌고있습니다...
2856 번째 루프를 돌고있습니다...
2857 번째 루프를 돌고있습니다...
2858 번째 루프를 돌고있습니다...
2859 번째 루프를 돌고있습니다...
2860 번째 루프를 돌고있습니다...
2861 번째 루프를 돌고있습니다...
2862 번째 루프를 돌고있습니다...
2863 번째 루프를 돌고있습니다...
2864 번째 루프를 돌고있습니다...
2865 번째 루프를 돌고있습니다...
2866 번째 루프를 돌고있습니다...
2867 번째 루프를 돌고있습니다...
2868 번째 루프를 돌고있습니다...
2869 번째 루프를 돌고있습니다...
2870 번째 루프를 돌고있습니다...
2871 번째 루프를 돌고있습니다...
2872 번째 루프를 돌고있습니다...
2873 번째 루프를 돌고있습니다...
2874 번째 루프를 돌고있습니다...
2875 번째 루프를 돌고있습니다...
2876 번째 루프를 돌고있습니다...
2877 번째 루프를 돌고있습니다...
2878 번째 루프를 돌고있습니다...
2879 번째 루프를 돌고있습니다...
2880 번째 루프를 돌고있습니다...
2881 번째 루프를 돌고있습니다...
2882 번째 루프를 돌고있습니다...
2883 번째 루프를 돌고있습니다...
2884 번째 루프를 돌고있습니다...
2885 번째 루프를 돌고있습니다...
2886 번째 루프를 돌고있습니다...
2887 번째 루프를 돌고있습니다...
2888 번째 루프를 돌고있습니다...
2889 번째 루프를 돌고있습니다...
2890 번째 루프를 돌고있습니다...
2891 번째 루프를 돌고있습니다...
2892 번째 루프를 돌고있습니다...
2893 번째 루프

3221 번째 루프를 돌고있습니다...
3222 번째 루프를 돌고있습니다...
3223 번째 루프를 돌고있습니다...
3224 번째 루프를 돌고있습니다...
3225 번째 루프를 돌고있습니다...
3226 번째 루프를 돌고있습니다...
3227 번째 루프를 돌고있습니다...
3228 번째 루프를 돌고있습니다...
3229 번째 루프를 돌고있습니다...
3230 번째 루프를 돌고있습니다...
3231 번째 루프를 돌고있습니다...
3232 번째 루프를 돌고있습니다...
3233 번째 루프를 돌고있습니다...
3234 번째 루프를 돌고있습니다...
3235 번째 루프를 돌고있습니다...
3236 번째 루프를 돌고있습니다...
3237 번째 루프를 돌고있습니다...
3238 번째 루프를 돌고있습니다...
3239 번째 루프를 돌고있습니다...
3240 번째 루프를 돌고있습니다...
3241 번째 루프를 돌고있습니다...
3242 번째 루프를 돌고있습니다...
3243 번째 루프를 돌고있습니다...
3244 번째 루프를 돌고있습니다...
3245 번째 루프를 돌고있습니다...
3246 번째 루프를 돌고있습니다...
3247 번째 루프를 돌고있습니다...
3248 번째 루프를 돌고있습니다...
3249 번째 루프를 돌고있습니다...
3250 번째 루프를 돌고있습니다...
3251 번째 루프를 돌고있습니다...
3252 번째 루프를 돌고있습니다...
3253 번째 루프를 돌고있습니다...
3254 번째 루프를 돌고있습니다...
3255 번째 루프를 돌고있습니다...
3256 번째 루프를 돌고있습니다...
3257 번째 루프를 돌고있습니다...
3258 번째 루프를 돌고있습니다...
3259 번째 루프를 돌고있습니다...
3260 번째 루프를 돌고있습니다...
3261 번째 루프를 돌고있습니다...
3262 번째 루프를 돌고있습니다...
3263 번째 루프를 돌고있습니다...
3264 번째 루프를 돌고있습니다...
3265 번째 루프를 돌고있습니다...
3266 번째 루프

In [64]:
final = pd.read_csv('final.csv')
final

,Unnamed: 0,date,firm,content,ng
0,0.0,2008-04-01,대우증권,2008_04 \n월간채권투자\n\n채 권 시 장 전 망\n\n對應과 限界\n\n...,미/NNG;국채/NNG;금리/NNG;안전자산/NNG;선호/NNG
1,0.0,2008-04-07,대우증권,Fixed Income Weekly \n2008. 4. 7. #918 \n\nFi...,글로벌/NNG;신용경색/NNG;완화/NNG
2,0.0,2008-04-07,대우증권,Fixed Income Weekly \n2008. 4. 7. #918 \n\nFi...,글로벌/NNG;신용경색/NNG;완화/NNG
3,0.0,2008-04-14,대우증권,총재의 변신은 무죄다?! \n\n \n \n \n\n 2008. 4. 14 \n...,예상/NNG;경제/NNG;성장률/NNG;하향/NNG
4,0.0,2008-04-21,대우증권,항상 열려는 있는 가능성 \n\n \n \n \n\n 2008. 4. 21 \n...,미/NNG;국채/NNG;금리/NNG;안전자산/NNG;선호/NNG
5,0.0,2008-04-28,대우증권,임박한 기회? \n\n \n \n \n\n 2008. 4. 28 \n\nFixe...,fed/NNG;원자재/NNG;가격/NNG;상승/NNG
6,0.0,2008-05-04,대우증권,2008_05 \n월간채권투자\n\n채 권 시 장 전 망\n\n機會의 再構成\n\...,미/NNG;국채/NNG;금리/NNG;안전자산/NNG;선호/NNG
7,0.0,2008-05-13,대우증권,\n \n \n\n환율의 경제학 \n\n 2008. 5. 13 \n\nFixe...,인하/NNG;기대/NNG;금리/NNG;하락/NNG
8,0.0,2008-05-19,대우증권,Daewoo Bond Brief \n\n \n\n2008년 5월 19일 \n\n윤일...,물가/NNG;상승/NNG;압력/NNG;금리/NNG;상승/NNG
9,0.0,2008-05-20,대우증권,Daewoo Bond Brief \n\n \n\n2008년 5월 20일 \n\n윤일...,금리/NNG;상승/NNG;환율/NNG;상승/NNG
